In [1]:
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame
from pandas import read_excel
from datetime import datetime,timedelta
from dateutil.relativedelta import relativedelta


In [2]:
# 수집할 컨텐츠가 있는 날짜별 웹 페이지 주소
url = "https://music.bugs.co.kr/chart/track/day/total"

# 브라우저 버전정보
userAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko)"

# 접속 객체 생성
# request는 직접 header 정보 갱신이 불가능해 session 생성
session = requests.Session()

# 접속 객체에 부가정보(header) 삽입, 크롤링이 가능하도록 유도
session.headers.update({
        "Referer": "",
        "User-Agent": userAgent,
})

# 생성한 접속 객체를 활용해 웹 페이지에 접속
# 지정된 URL에 GET 요청을 보내고
# 그에 대한 응답을 r 변수에 할당
r = session.get(url)

# 접속 실패 시 에러 코드와 에러 메시지 출력
if r.status_code != 200:
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
    raise Exception(msg)
        
# 인코딩 형식을 지정해 beautifulsoup 객체를 생성
r.encoding = "utf-8"

# BeautifulSoup 객체 soup를 생성
soup = BeautifulSoup(r.content,"html.parser")


In [3]:
# dict 형식의 제목, 가수, 앨범, 순위, 차트 일수 등 정보를 담을 빈 list 생성
일간차트 = []

#기준 날짜 설정
query = datetime(2023,6,1)
now = datetime.now()
nowday = now.strftime('%Y%m%d')
days = int(nowday)

#기준 날짜부터 오늘까지 반복
while query <= datetime.strptime(nowday, '%Y%m%d'):
    #벅스 홈페이지 URL
    url = "https://music.bugs.co.kr/chart/track/day/total?chartdate={}".format(query.strftime('%Y%m%d'))
    r = session.get(url)
    soup = BeautifulSoup(r.content,"html.parser")

    #각 음악 정보 추출하여 빈 리스트에 추가
    titles = soup.select("p.title") #노래명 
    artists = soup.select("p.artist") #가수명
    albums = soup.select("a.album") #앨범명

    for i in range(len(titles)):
        title = titles[i].text.strip()
        artist = artists[i].text.strip()
        album = albums[i+1].text.strip()
        rank = i+1 
        if rank ==1:
            days-=1

        #음악 정보를 딕셔너리로 저장하여 리스트에 추가
        chart_list = {"곡명": title, "아티스트":artist, "앨범명":album, "순위" : rank, "날짜": query.strftime('%Y%m%d')}
        일간차트.append(chart_list)
    #다음 날짜로 이동
    query += timedelta(days=1)
df = DataFrame(일간차트)
# 순위를 우선해서 보여주기 위해 index로 설정
df.set_index("순위", inplace=True)
df.to_excel("벅스일간차트.xlsx")
df

,곡명,아티스트,앨범명,날짜
순위,,,,
1,퀸카 (Queencard),(여자)아이들,I feel,20230601
2,Spicy,aespa,MY WORLD - The 3rd Mini Album,20230601
3,I AM,IVE (아이브),I've IVE,20230601
4,UNFORGIVEN (feat. Nile Rodgers),LE SSERAFIM (르세라핌),UNFORGIVEN,20230601
5,Kitsch,IVE (아이브),I've IVE,20230601
...,...,...,...,...
96,Poppy (Korean Ver.),STAYC(스테이씨),Teddy Bear,20230629
97,불면증 (Insomnia),SHINee (샤이니),HARD - The 8th Album,20230629
98,새삥 (Prod. ZICO) (Feat. 호미들),지코 (ZICO),스트릿 맨 파이터(SMF) Original Vol.3 (계급미션),20230629
